In [15]:
import pandas as pd
import numpy as np
import yfinance as yf
import re
from fuzzywuzzy import fuzz
from countrygroups import EUROPEAN_UNION

# https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard 

In [17]:
def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x


def get_working_capital(zone=""):
    path = "data_damodaran/cash_flow_estimation/working_capital/wcdata" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_historical(zone=""):
    path = "data_damodaran/growth/historical_growth/histgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_capex(zone=""):
    path = "data_damodaran/cash_flow_estimation/CAPEX/capex" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x


def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x


def get_growth_ebit(zone=""):
    path = "data_damodaran/growth/growth_ebit/fundgrEB" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

industries = list(get_capex().index)

def get_industry(ticket):
    ls = []
    ind = yf.Ticker(ticket).info.get("industry")
    for i in industries:
        ls.append([fuzz.ratio(ind, i),i])
    return pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]

def zones(x):
    if x == "United States":
        return ""
    if x == "China":
        return "China"
    if x == "Canada":
        return "emerg"
    if x == "Australia":
        return "emerg"
    if x == "New Zeland":
        return "emerg"
    if x in EUROPEAN_UNION.names:
        return "Europe"
    if x == "India":
        return "India"
    if x == "Japan":
        return "Japan"
    else:
        return "Global"

In [14]:
%%time
n = 4
x = yf.Ticker("BABA")
sector = "Software (System & Application)"
#x.info#.get('industry')
x.info["country"]

CPU times: user 12.3 ms, sys: 3.58 ms, total: 15.9 ms
Wall time: 278 ms


'China'

In [18]:
def freecashflow(ticket,n=4):
    x = yf.Ticker(ticket)
    sector = get_industry(ticket)
    zone = zones(x.info["country"])
    
    ebit = x.income_stmt.loc["EBIT",:].sort_index()
    g_ebit = get_growth_ebit(zone).loc[sector,"Expected Growth in EBIT"]
    ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g_ebit)**i) for i in range(n+1)], index=range(0,n+1))[1:]

    tax_rate = get_taxrates(zone).loc[sector, "Average across only money-making companies"]

    usa_ktn_percentage = get_working_capital(zone).loc[sector,"Non-cash WC/ Sales"]
    sales_growth_2y = get_growth_historical(zone).loc[sector,"Expected Growth in Revenues - Next 2 years"]
    g_sales = (1+sales_growth_2y)**(1/2)-1

    sales = x.incomestmt.loc["Total Revenue",:].sort_index()
    sales_forecast = pd.Series([sales.iloc[-1]*((1+g_sales)**i) for i in range(n+1)], index=range(0,n+1))
    sales_forecast

    ktn_forecast = sales_forecast * usa_ktn_percentage
    ktn_delta_forecast = ktn_forecast.diff()[1:]

    capex_vs_sales = get_capex(zone).loc[sector, "Net Cap Ex/Sales"]
    capex_net = (sales_forecast * capex_vs_sales)[1:]


    return ebit_forecast*(1-tax_rate) - capex_net - ktn_delta_forecast

get_industry("MSFT")


'Software (Internet)'

In [3]:
def freecashflow(ticket,sector,n=4):
    x = yf.Ticker(ticket)
    ebit = x.income_stmt.loc["EBIT",:].sort_index()
    g_ebit = get_growth_ebit("").loc[sector,"Expected Growth in EBIT"]
    ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g_ebit)**i) for i in range(n+1)], index=range(0,n+1))[1:]

    tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]

    usa_ktn_percentage = get_working_capital("").loc[sector,"Non-cash WC/ Sales"]
    sales_growth_2y = get_growth_historical("").loc[sector,"Expected Growth in Revenues - Next 2 years"]
    g_sales = (1+sales_growth_2y)**(1/2)-1

    sales = x.incomestmt.loc["Total Revenue",:].sort_index()
    sales_forecast = pd.Series([sales.iloc[-1]*((1+g_sales)**i) for i in range(n+1)], index=range(0,n+1))
    sales_forecast

    ktn_forecast = sales_forecast * usa_ktn_percentage
    ktn_delta_forecast = ktn_forecast.diff()[1:]

    capex_vs_sales = get_capex("").loc[sector, "Net Cap Ex/Sales"]
    capex_net = (sales_forecast * capex_vs_sales)[1:]


    return ebit_forecast*(1-tax_rate) - capex_net - ktn_delta_forecast

freecashflow("MSFT", "Software (System & Application)")

1    4.237772e+10
2    6.620163e+10
3    9.853806e+10
4    1.421420e+11
dtype: float64

In [6]:
yf.Ticker("MSFT").info.get("industry")


'Software - Infrastructure'

In [75]:
freecashflow("MSFT", "Software (System & Application)")

yf.Ticker("KO").info.get("industry")

list(get_capex("").index)
yf.Ticker("KO").info.get("industry")

freecashflow("META", "Beverage (Soft)")
yf.Ticker("META").info.get("industry")

freecashflow("MSFT", "Software (System & Application)")

1    4.237772e+10
2    6.620163e+10
3    9.853806e+10
4    1.421420e+11
dtype: float64

In [53]:
g_ebit = get_growth_ebit("").loc[sector,"Expected Growth in EBIT"]
ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g_ebit)**i) for i in range(n+1)], index=range(0,n+1))[1:]
g_ebit

0.30928579000938206